## GEOGRAPHICAL FEATURE VISUALIZATION

In [41]:
import folium
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from folium import Choropleth , Circle , Marker , Icon ,Map
from folium.plugins import HeatMap, MarkerCluster

In [105]:
claimsdf = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/ProyectoFinal/Claims-Frequency-Predictions/Notebooks/claimsdf_2.csv')

In [106]:
claimsdf.head()

,ClaimNb,Exposure,Area,BonusMalus,VehBrand,Density,Region,empirical_frequencies,VehGas_Regular,VehPower_,VehAge_,DrivAge_
0,1,0.10,4,50,9,1217,Auvergne-Rhône-Alpes,10.000000,1,5,1,6
1,1,0.77,4,50,9,1217,Auvergne-Rhône-Alpes,1.298701,1,5,1,6
2,1,0.75,2,50,9,54,Hauts-de-France,1.333333,0,6,2,6
3,1,0.09,2,50,9,76,Nouvelle-Aquitaine,11.111111,0,7,1,5
4,1,0.84,2,50,9,76,Nouvelle-Aquitaine,1.190476,0,7,1,5


### GEOS

In [76]:
geo_json = r'regions.geojson'

with open (geo_json) as geo_file:
    geo_france =  json.load(geo_file)

In [68]:
"""
#gdf = gpd.GeoDataFrame.from_features(geo_france["features"])
#O..

gdf.to_file('regions.geojson',driver='GeoJSON')

gdf.crs = "EPSG:4326"
gdf = gdf.to_crs("EPSG:3857")
print(gdf.crs)
"""

In [83]:
geo_france["features"][1]["properties"]

{'code': '24', 'nom': 'Centre-Val de Loire'}

In [77]:
for i in range(len(geo_france.get("features"))):
    print(geo_france.get("features")[i].get("properties"))

{'code': '11', 'nom': 'Île-de-France'}
{'code': '24', 'nom': 'Centre-Val de Loire'}
{'code': '27', 'nom': 'Bourgogne-Franche-Comté'}
{'code': '28', 'nom': 'Normandie'}
{'code': '32', 'nom': 'Hauts-de-France'}
{'code': '44', 'nom': 'Grand Est'}
{'code': '52', 'nom': 'Pays de la Loire'}
{'code': '53', 'nom': 'Bretagne'}
{'code': '75', 'nom': 'Nouvelle-Aquitaine'}
{'code': '76', 'nom': 'Occitanie'}
{'code': '84', 'nom': 'Auvergne-Rhône-Alpes'}
{'code': '93', 'nom': "Provence-Alpes-Côte d'Azur"}
{'code': '94', 'nom': 'Corse'}


In [78]:
france_regions_map = folium.Map(location=[46.72213865950384,2.6911389975442024], tiles='CartoDB dark_matter', zoom_start=6)
france_regions_map

In [84]:

france_regions_map.choropleth(
    geo_data = geo_json, 
    data = claimsdf, 
    columns = ["Region", "Density"], 
    key_on = "features.properties.nom", 
    fill_color = "YlGnBu", 
    fill_opacity = 1, 
    line_opacity = 1, 
    legend_name = "Regions", 
    smooth_factor = 0)

france_regions_map



"""
comunidades_mapa.choropleth(
    geo_data = geo_json, 
    data = fp, 
    columns = ["province", "pred_cluster"], 
    key_on = "feature.properties.name", 
    fill_color = "YlGnBu", 
    fill_opacity = 1, 
    line_opacity = 1, 
    legend_name = "Clusters", 
    smooth_factor = 0)

comunidades_mapa
"""

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
france_regions_map.save("choro.jpg")

#### Now, we proceed to export a csv file with the wrangled data frame for mapping

In [44]:
claimsdf.to_csv('claimsdf_3.csv', index = False)